### Imports

In [1]:
import os, time, pickle, random, time
from datetime import datetime
import numpy as np
from time import localtime, strftime
import logging, scipy
import tensorflow as tf
import tensorlayer as tl
from model import SRGAN_g, SRGAN_d, Vgg19_simple_api
from utils import *
from config import config, log_config

### Hyper Params

In [2]:
batch_size = config.TRAIN.batch_size
lr_init = config.TRAIN.lr_init
beta1 = config.TRAIN.beta1
n_epoch_init = config.TRAIN.n_epoch_init

### Dirs and Stuff

In [3]:
## create folders to save result images and trained model
save_dir_ginit = "samples/{}_ginit".format("srgan")
save_dir_gan = "samples/{}_gan".format("srgan")
tl.files.exists_or_mkdir(save_dir_ginit)
tl.files.exists_or_mkdir(save_dir_gan)
checkpoint_dir = "checkpoint"  # checkpoint_resize_conv
tl.files.exists_or_mkdir(checkpoint_dir)

[TL] [!] samples/srgan_ginit exists ...
[TL] [!] samples/srgan_gan exists ...
[TL] [!] checkpoint exists ...


True

### Preload Data because AWS

In [4]:
train_hr_img_list = sorted(tl.files.load_file_list(path=config.TRAIN.hr_img_path, regx='.*.png', printable=False))
train_lr_img_list = sorted(tl.files.load_file_list(path=config.TRAIN.lr_img_path, regx='.*.png', printable=False))
valid_hr_img_list = sorted(tl.files.load_file_list(path=config.VALID.hr_img_path, regx='.*.png', printable=False))
valid_lr_img_list = sorted(tl.files.load_file_list(path=config.VALID.lr_img_path, regx='.*.png', printable=False))

In [5]:
train_hr_imgs = tl.vis.read_images(train_hr_img_list, path=config.TRAIN.hr_img_path, n_threads=32)

[TL] read 14 from trainData/HR/


In [ ]:
ni = int(np.sqrt(batch_size))
sample_imgs = train_hr_imgs[0:batch_size]
sample_imgs = tl.vis.read_images(train_hr_img_list[0:batch_size], path=config.TRAIN.hr_img_path, n_threads=32) # if no pre-load train set
sample_imgs_384 = tl.prepro.threading_data(sample_imgs, fn=crop_sub_imgs_fn, is_random=False)
print('sample HR sub-image:', sample_imgs_384[0].shape, sample_imgs_384[0].min(), sample_imgs_384[0].max())
sample_imgs_96 = tl.prepro.threading_data(sample_imgs_384, fn=downsample_fn)
print('sample LR sub-image:', sample_imgs_96[0].shape, sample_imgs_96[0].min(), sample_imgs_96[0].max())
tl.vis.save_images(sample_imgs_96[0], [ni, ni], save_dir_ginit + '/_train_sample_96.png')
tl.vis.save_images(sample_imgs_384[0], [ni, ni], save_dir_ginit + '/_train_sample_384.png')
tl.vis.save_images(sample_imgs_96, [ni, ni], save_dir_gan + '/_train_sample_96.png')
tl.vis.save_images(sample_imgs_384[0], [ni, ni], save_dir_gan + '/_train_sample_384.png')

In [6]:
for im in train_hr_imgs:
    print(im.shape)

(480, 500, 3)
(576, 720, 3)
(512, 512)
(288, 352, 3)
(360, 248, 3)
(276, 276, 3)
(360, 500, 3)
(288, 352, 3)
(512, 512, 3)
(512, 512, 3)
(512, 768, 3)
(512, 512, 3)
(656, 528, 3)
(388, 584, 3)


In [7]:
train_lr_img = tl.vis.read_images(train_lr_img_list, path=config.TRAIN.lr_img_path, n_threads=32)

[TL] read 14 from trainData/LR/


In [8]:
# Some outputs are lower than the default for set14 images, therefore check or else throws a crop error
for im in train_lr_img:
    print(im.shape)

(120, 125, 3)
(144, 180, 3)
(128, 128)
(72, 88, 3)
(90, 62, 3)
(69, 69, 3)
(90, 125, 3)
(72, 88, 3)
(128, 128, 3)
(128, 128, 3)
(128, 192, 3)
(128, 128, 3)
(164, 132, 3)
(97, 146, 3)


In [9]:
t_image = tf.placeholder('float32', [batch_size, 62, 62, 3], name='t_image_input_to_SRGAN_generator')

In [10]:
t_target_image = tf.placeholder('float32', [batch_size, 248, 248, 3], name='t_target_image')

In [11]:
net_g = SRGAN_g(t_image, is_train=True, reuse=False)

[TL] InputLayer  SRGAN_g/in: (4, 62, 62, 3)
[TL] Conv2d SRGAN_g/n64s1/c: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
Instructions for updating:
Colocations handled automatically by placer.
[TL] Conv2d SRGAN_g/n64s1/c1/0: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b1/0: decay: 0.900000 epsilon: 0.000010 act: relu is_train: True
[TL] Conv2d SRGAN_g/n64s1/c2/0: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b2/0: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: True
[TL] ElementwiseLayer SRGAN_g/b_residual_add/0: size: (4, 62, 62, 64) fn: add
[TL] Conv2d SRGAN_g/n64s1/c1/1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b1/1: decay: 0.900000 epsilon: 0.000010 act: relu is_train: True
[TL] Conv2d SRGAN_g/n64s1/c2/1: n_filter: 64 filter_size: (3, 3) strides

[TL] Conv2d SRGAN_g/n64s1/c2/15: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b2/15: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: True
[TL] ElementwiseLayer SRGAN_g/b_residual_add/15: size: (4, 62, 62, 64) fn: add
[TL] Conv2d SRGAN_g/n64s1/c/m: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b/m: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: True
[TL] ElementwiseLayer SRGAN_g/add3: size: (4, 62, 62, 64) fn: add
[TL] Conv2d SRGAN_g/n256s1/1: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] SubpixelConv2d  SRGAN_g/pixelshufflerx2/1: scale: 2 n_out_channel: 64 act: relu
[TL] Conv2d SRGAN_g/n256s1/2: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] SubpixelConv2d  SRGAN_g/pixelshufflerx2/2: scale: 2 n_out_channel: 64 act: relu
[TL] Conv2d SRGAN_g/out: n_filt

In [12]:
net_d, logits_real = SRGAN_d(t_target_image, is_train=True, reuse=False)

[TL] WARNING: Function: `tensorlayer.layers.utils.set_name_reuse` (in file: /home/ubuntu/anaconda3/envs/srgan/lib/python3.7/site-packages/tensorlayer/layers/utils.py) is deprecated and will be removed after 2018-06-30.
Instructions for updating: TensorLayer relies on TensorFlow to check name reusing

[TL] WARNING: this method is DEPRECATED and has no effect, please remove it from your code.
[TL] InputLayer  SRGAN_d/input/images: (4, 248, 248, 3)
[TL] Conv2d SRGAN_d/h0/c: n_filter: 64 filter_size: (4, 4) strides: (2, 2) pad: SAME act: <lambda>
[TL] WARNING: Function: `tensorlayer.activation.leaky_relu` (in file: /home/ubuntu/anaconda3/envs/srgan/lib/python3.7/site-packages/tensorlayer/activation.py) is deprecated and will be removed after 2018-09-30.
Instructions for updating: This API is deprecated. Please use as `tf.nn.leaky_relu`

[TL] Conv2d SRGAN_d/h1/c: n_filter: 128 filter_size: (4, 4) strides: (2, 2) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_d/h1/bn: decay: 0.900000

In [13]:
_, logits_fake = SRGAN_d(net_g.outputs, is_train=True, reuse=True)

[TL] WARNING: this method is DEPRECATED and has no effect, please remove it from your code.
[TL] InputLayer  SRGAN_d/input/images: (4, 248, 248, 3)
[TL] Conv2d SRGAN_d/h0/c: n_filter: 64 filter_size: (4, 4) strides: (2, 2) pad: SAME act: <lambda>
[TL] Conv2d SRGAN_d/h1/c: n_filter: 128 filter_size: (4, 4) strides: (2, 2) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_d/h1/bn: decay: 0.900000 epsilon: 0.000010 act: <lambda> is_train: True
[TL] Conv2d SRGAN_d/h2/c: n_filter: 256 filter_size: (4, 4) strides: (2, 2) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_d/h2/bn: decay: 0.900000 epsilon: 0.000010 act: <lambda> is_train: True
[TL] Conv2d SRGAN_d/h3/c: n_filter: 512 filter_size: (4, 4) strides: (2, 2) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_d/h3/bn: decay: 0.900000 epsilon: 0.000010 act: <lambda> is_train: True
[TL] Conv2d SRGAN_d/h4/c: n_filter: 1024 filter_size: (4, 4) strides: (2, 2) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_d/h4/bn: dec

In [14]:
net_g.print_params(False)
net_g.print_layers()
net_d.print_params(False)
net_d.print_layers()

[TL]   param   0: SRGAN_g/n64s1/c/kernel:0 (3, 3, 3, 64)      float32_ref
[TL]   param   1: SRGAN_g/n64s1/c/bias:0 (64,)              float32_ref
[TL]   param   2: SRGAN_g/n64s1/c1/0/kernel:0 (3, 3, 64, 64)     float32_ref
[TL]   param   3: SRGAN_g/n64s1/b1/0/beta:0 (64,)              float32_ref
[TL]   param   4: SRGAN_g/n64s1/b1/0/gamma:0 (64,)              float32_ref
[TL]   param   5: SRGAN_g/n64s1/b1/0/moving_mean:0 (64,)              float32_ref
[TL]   param   6: SRGAN_g/n64s1/b1/0/moving_variance:0 (64,)              float32_ref
[TL]   param   7: SRGAN_g/n64s1/c2/0/kernel:0 (3, 3, 64, 64)     float32_ref
[TL]   param   8: SRGAN_g/n64s1/b2/0/beta:0 (64,)              float32_ref
[TL]   param   9: SRGAN_g/n64s1/b2/0/gamma:0 (64,)              float32_ref
[TL]   param  10: SRGAN_g/n64s1/b2/0/moving_mean:0 (64,)              float32_ref
[TL]   param  11: SRGAN_g/n64s1/b2/0/moving_variance:0 (64,)              float32_ref
[TL]   param  12: SRGAN_g/n64s1/c1/1/kernel:0 (3, 3, 64, 64)  

[TL]   param 104: SRGAN_g/n64s1/b1/10/gamma:0 (64,)              float32_ref
[TL]   param 105: SRGAN_g/n64s1/b1/10/moving_mean:0 (64,)              float32_ref
[TL]   param 106: SRGAN_g/n64s1/b1/10/moving_variance:0 (64,)              float32_ref
[TL]   param 107: SRGAN_g/n64s1/c2/10/kernel:0 (3, 3, 64, 64)     float32_ref
[TL]   param 108: SRGAN_g/n64s1/b2/10/beta:0 (64,)              float32_ref
[TL]   param 109: SRGAN_g/n64s1/b2/10/gamma:0 (64,)              float32_ref
[TL]   param 110: SRGAN_g/n64s1/b2/10/moving_mean:0 (64,)              float32_ref
[TL]   param 111: SRGAN_g/n64s1/b2/10/moving_variance:0 (64,)              float32_ref
[TL]   param 112: SRGAN_g/n64s1/c1/11/kernel:0 (3, 3, 64, 64)     float32_ref
[TL]   param 113: SRGAN_g/n64s1/b1/11/beta:0 (64,)              float32_ref
[TL]   param 114: SRGAN_g/n64s1/b1/11/gamma:0 (64,)              float32_ref
[TL]   param 115: SRGAN_g/n64s1/b1/11/moving_mean:0 (64,)              float32_ref
[TL]   param 116: SRGAN_g/n64s1/b1/11/

[TL]   layer  37: SRGAN_g/n64s1/c1/7/Conv2D:0 (4, 62, 62, 64)    float32
[TL]   layer  38: SRGAN_g/n64s1/b1/7/Relu:0 (4, 62, 62, 64)    float32
[TL]   layer  39: SRGAN_g/n64s1/c2/7/Conv2D:0 (4, 62, 62, 64)    float32
[TL]   layer  40: SRGAN_g/n64s1/b2/7/batchnorm/Add_1:0 (4, 62, 62, 64)    float32
[TL]   layer  41: SRGAN_g/b_residual_add/7:0 (4, 62, 62, 64)    float32
[TL]   layer  42: SRGAN_g/n64s1/c1/8/Conv2D:0 (4, 62, 62, 64)    float32
[TL]   layer  43: SRGAN_g/n64s1/b1/8/Relu:0 (4, 62, 62, 64)    float32
[TL]   layer  44: SRGAN_g/n64s1/c2/8/Conv2D:0 (4, 62, 62, 64)    float32
[TL]   layer  45: SRGAN_g/n64s1/b2/8/batchnorm/Add_1:0 (4, 62, 62, 64)    float32
[TL]   layer  46: SRGAN_g/b_residual_add/8:0 (4, 62, 62, 64)    float32
[TL]   layer  47: SRGAN_g/n64s1/c1/9/Conv2D:0 (4, 62, 62, 64)    float32
[TL]   layer  48: SRGAN_g/n64s1/b1/9/Relu:0 (4, 62, 62, 64)    float32
[TL]   layer  49: SRGAN_g/n64s1/c2/9/Conv2D:0 (4, 62, 62, 64)    float32
[TL]   layer  50: SRGAN_g/n64s1/b2/9/batc

[TL]   layer   3: SRGAN_d/h1/bn/leaky_relu:0 (4, 62, 62, 128)    float32
[TL]   layer   4: SRGAN_d/h2/c/Conv2D:0 (4, 31, 31, 256)    float32
[TL]   layer   5: SRGAN_d/h2/bn/leaky_relu:0 (4, 31, 31, 256)    float32
[TL]   layer   6: SRGAN_d/h3/c/Conv2D:0 (4, 16, 16, 512)    float32
[TL]   layer   7: SRGAN_d/h3/bn/leaky_relu:0 (4, 16, 16, 512)    float32
[TL]   layer   8: SRGAN_d/h4/c/Conv2D:0 (4, 8, 8, 1024)    float32
[TL]   layer   9: SRGAN_d/h4/bn/leaky_relu:0 (4, 8, 8, 1024)    float32
[TL]   layer  10: SRGAN_d/h5/c/Conv2D:0 (4, 4, 4, 2048)    float32
[TL]   layer  11: SRGAN_d/h5/bn/leaky_relu:0 (4, 4, 4, 2048)    float32
[TL]   layer  12: SRGAN_d/h6/c/Conv2D:0 (4, 4, 4, 1024)    float32
[TL]   layer  13: SRGAN_d/h6/bn/leaky_relu:0 (4, 4, 4, 1024)    float32
[TL]   layer  14: SRGAN_d/h7/c/Conv2D:0 (4, 4, 4, 512)     float32
[TL]   layer  15: SRGAN_d/h7/bn/batchnorm/Add_1:0 (4, 4, 4, 512)     float32
[TL]   layer  16: SRGAN_d/res/c/Conv2D:0 (4, 4, 4, 128)     float32
[TL]   layer  17

In [15]:
t_target_image_224 = tf.image.resize_images(t_target_image, size=[224, 224], method=0,align_corners=False)

In [16]:
t_predict_image_224 = tf.image.resize_images(net_g.outputs, size=[224, 224], method=0, align_corners=False)

In [17]:
net_vgg, vgg_target_emb = Vgg19_simple_api((t_target_image_224 + 1) / 2, reuse=False)
_, vgg_predict_emb = Vgg19_simple_api((t_predict_image_224 + 1) / 2, reuse=True)

build model started
[TL] InputLayer  VGG19/input: (4, 224, 224, 3)
[TL] Conv2d VGG19/conv1_1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d VGG19/conv1_2: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] MaxPool2d VGG19/pool1: filter_size: (2, 2) strides: (2, 2) padding: SAME
Instructions for updating:
Use keras.layers.max_pooling2d instead.
[TL] Conv2d VGG19/conv2_1: n_filter: 128 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d VGG19/conv2_2: n_filter: 128 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] MaxPool2d VGG19/pool2: filter_size: (2, 2) strides: (2, 2) padding: SAME
[TL] Conv2d VGG19/conv3_1: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d VGG19/conv3_2: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d VGG19/conv3_3: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d VGG19/conv3_4: n_filt

In [18]:
## test inference
net_g_test = SRGAN_g(t_image, is_train=False, reuse=True)

[TL] InputLayer  SRGAN_g/in: (4, 62, 62, 3)
[TL] Conv2d SRGAN_g/n64s1/c: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d SRGAN_g/n64s1/c1/0: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b1/0: decay: 0.900000 epsilon: 0.000010 act: relu is_train: False
[TL] Conv2d SRGAN_g/n64s1/c2/0: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b2/0: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: False
[TL] ElementwiseLayer SRGAN_g/b_residual_add/0: size: (4, 62, 62, 64) fn: add
[TL] Conv2d SRGAN_g/n64s1/c1/1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b1/1: decay: 0.900000 epsilon: 0.000010 act: relu is_train: False
[TL] Conv2d SRGAN_g/n64s1/c2/1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64

[TL] ElementwiseLayer SRGAN_g/b_residual_add/15: size: (4, 62, 62, 64) fn: add
[TL] Conv2d SRGAN_g/n64s1/c/m: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b/m: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: False
[TL] ElementwiseLayer SRGAN_g/add3: size: (4, 62, 62, 64) fn: add
[TL] Conv2d SRGAN_g/n256s1/1: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] SubpixelConv2d  SRGAN_g/pixelshufflerx2/1: scale: 2 n_out_channel: 64 act: relu
[TL] Conv2d SRGAN_g/n256s1/2: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] SubpixelConv2d  SRGAN_g/pixelshufflerx2/2: scale: 2 n_out_channel: 64 act: relu
[TL] Conv2d SRGAN_g/out: n_filter: 3 filter_size: (1, 1) strides: (1, 1) pad: SAME act: tanh


In [19]:
# ###========================== DEFINE TRAIN OPS ==========================###
d_loss1 = tl.cost.sigmoid_cross_entropy(logits_real, tf.ones_like(logits_real), name='d1')
d_loss2 = tl.cost.sigmoid_cross_entropy(logits_fake, tf.zeros_like(logits_fake), name='d2')
d_loss = d_loss1 + d_loss2

g_gan_loss = 1e-3 * tl.cost.sigmoid_cross_entropy(logits_fake, tf.ones_like(logits_fake), name='g')
mse_loss = tl.cost.mean_squared_error(net_g.outputs, t_target_image, is_mean=True)
vgg_loss = 2e-6 * tl.cost.mean_squared_error(vgg_predict_emb.outputs, vgg_target_emb.outputs, is_mean=True)

g_loss = mse_loss + vgg_loss + g_gan_loss

g_vars = tl.layers.get_variables_with_name('SRGAN_g', True, True)
d_vars = tl.layers.get_variables_with_name('SRGAN_d', True, True)

with tf.variable_scope('learning_rate'):
    lr_v = tf.Variable(lr_init, trainable=False)

[TL]   [*] geting variables with SRGAN_g
[TL]   got   0: SRGAN_g/n64s1/c/kernel:0   (3, 3, 3, 64)
[TL]   got   1: SRGAN_g/n64s1/c/bias:0   (64,)
[TL]   got   2: SRGAN_g/n64s1/c1/0/kernel:0   (3, 3, 64, 64)
[TL]   got   3: SRGAN_g/n64s1/b1/0/beta:0   (64,)
[TL]   got   4: SRGAN_g/n64s1/b1/0/gamma:0   (64,)
[TL]   got   5: SRGAN_g/n64s1/c2/0/kernel:0   (3, 3, 64, 64)
[TL]   got   6: SRGAN_g/n64s1/b2/0/beta:0   (64,)
[TL]   got   7: SRGAN_g/n64s1/b2/0/gamma:0   (64,)
[TL]   got   8: SRGAN_g/n64s1/c1/1/kernel:0   (3, 3, 64, 64)
[TL]   got   9: SRGAN_g/n64s1/b1/1/beta:0   (64,)
[TL]   got  10: SRGAN_g/n64s1/b1/1/gamma:0   (64,)
[TL]   got  11: SRGAN_g/n64s1/c2/1/kernel:0   (3, 3, 64, 64)
[TL]   got  12: SRGAN_g/n64s1/b2/1/beta:0   (64,)
[TL]   got  13: SRGAN_g/n64s1/b2/1/gamma:0   (64,)
[TL]   got  14: SRGAN_g/n64s1/c1/2/kernel:0   (3, 3, 64, 64)
[TL]   got  15: SRGAN_g/n64s1/b1/2/beta:0   (64,)
[TL]   got  16: SRGAN_g/n64s1/b1/2/gamma:0   (64,)
[TL]   got  17: SRGAN_g/n64s1/c2/2/kernel:0  

In [20]:
g_optim_init = tf.train.AdamOptimizer(lr_v, beta1=beta1).minimize(mse_loss, var_list=g_vars)
## SRGAN
g_optim = tf.train.AdamOptimizer(lr_v, beta1=beta1).minimize(g_loss, var_list=g_vars)
d_optim = tf.train.AdamOptimizer(lr_v, beta1=beta1).minimize(d_loss, var_list=d_vars)

Instructions for updating:
Use tf.cast instead.


In [21]:
###========================== RESTORE MODEL =============================###
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False))
tl.layers.initialize_global_variables(sess)
if tl.files.load_and_assign_npz(sess=sess, name=checkpoint_dir + '/g_{}.npz'.format("srgan"), network=net_g) is False:
    tl.files.load_and_assign_npz(sess=sess, name=checkpoint_dir + '/g_{}_init.npz'.format("srgan"), network=net_g)
tl.files.load_and_assign_npz(sess=sess, name=checkpoint_dir + '/d_{}.npz'.format("srgan"), network=net_d)

[TL] WARNING: Function: `tensorlayer.layers.utils.initialize_global_variables` (in file: /home/ubuntu/anaconda3/envs/srgan/lib/python3.7/site-packages/tensorlayer/layers/utils.py) is deprecated and will be removed after 2018-09-30.
Instructions for updating: This API is deprecated in favor of `tf.global_variables_initializer`

[TL] ERROR: file checkpoint/g_srgan.npz doesn't exist.
[TL] ERROR: file checkpoint/d_srgan.npz doesn't exist.


In [22]:
###============================= LOAD VGG ===============================###
vgg19_npy_path = "../vgg19.npy"
if not os.path.isfile(vgg19_npy_path):
    print("Please download vgg19.npz from : https://github.com/machrisaa/tensorflow-vgg")
    exit()
npz = np.load(vgg19_npy_path, encoding='latin1').item()

params = []
for val in sorted(npz.items()):
    W = np.asarray(val[1][0])
    b = np.asarray(val[1][1])
    print("  Loading %s: %s, %s" % (val[0], W.shape, b.shape))
    params.extend([W, b])
tl.files.assign_params(sess, params, net_vgg)
# net_vgg.print_params(False)
# net_vgg.print_layers()

  Loading conv1_1: (3, 3, 3, 64), (64,)
  Loading conv1_2: (3, 3, 64, 64), (64,)
  Loading conv2_1: (3, 3, 64, 128), (128,)
  Loading conv2_2: (3, 3, 128, 128), (128,)
  Loading conv3_1: (3, 3, 128, 256), (256,)
  Loading conv3_2: (3, 3, 256, 256), (256,)
  Loading conv3_3: (3, 3, 256, 256), (256,)
  Loading conv3_4: (3, 3, 256, 256), (256,)
  Loading conv4_1: (3, 3, 256, 512), (512,)
  Loading conv4_2: (3, 3, 512, 512), (512,)
  Loading conv4_3: (3, 3, 512, 512), (512,)
  Loading conv4_4: (3, 3, 512, 512), (512,)
  Loading conv5_1: (3, 3, 512, 512), (512,)
  Loading conv5_2: (3, 3, 512, 512), (512,)
  Loading conv5_3: (3, 3, 512, 512), (512,)
  Loading conv5_4: (3, 3, 512, 512), (512,)
  Loading fc6: (25088, 4096), (4096,)
  Loading fc7: (4096, 4096), (4096,)
  Loading fc8: (4096, 1000), (1000,)


[<tf.Tensor 'Assign:0' shape=(3, 3, 3, 64) dtype=float32_ref>,
 <tf.Tensor 'Assign_1:0' shape=(64,) dtype=float32_ref>,
 <tf.Tensor 'Assign_2:0' shape=(3, 3, 64, 64) dtype=float32_ref>,
 <tf.Tensor 'Assign_3:0' shape=(64,) dtype=float32_ref>,
 <tf.Tensor 'Assign_4:0' shape=(3, 3, 64, 128) dtype=float32_ref>,
 <tf.Tensor 'Assign_5:0' shape=(128,) dtype=float32_ref>,
 <tf.Tensor 'Assign_6:0' shape=(3, 3, 128, 128) dtype=float32_ref>,
 <tf.Tensor 'Assign_7:0' shape=(128,) dtype=float32_ref>,
 <tf.Tensor 'Assign_8:0' shape=(3, 3, 128, 256) dtype=float32_ref>,
 <tf.Tensor 'Assign_9:0' shape=(256,) dtype=float32_ref>,
 <tf.Tensor 'Assign_10:0' shape=(3, 3, 256, 256) dtype=float32_ref>,
 <tf.Tensor 'Assign_11:0' shape=(256,) dtype=float32_ref>,
 <tf.Tensor 'Assign_12:0' shape=(3, 3, 256, 256) dtype=float32_ref>,
 <tf.Tensor 'Assign_13:0' shape=(256,) dtype=float32_ref>,
 <tf.Tensor 'Assign_14:0' shape=(3, 3, 256, 256) dtype=float32_ref>,
 <tf.Tensor 'Assign_15:0' shape=(256,) dtype=float32_re

In [ ]:
sess.run(tf.assign(lr_v, lr_init))
print(" ** fixed learning rate: %f (for init G)" % lr_init)
for epoch in range(0, n_epoch_init + 1):
    epoch_time = time.time()
    total_mse_loss, n_iter = 0, 0

    ## If your machine cannot load all images into memory, you should use
    ## this one to load batch of images while training.
    # random.shuffle(train_hr_img_list)
    # for idx in range(0, len(train_hr_img_list), batch_size):
    #     step_time = time.time()
    #     b_imgs_list = train_hr_img_list[idx : idx + batch_size]
    #     b_imgs = tl.prepro.threading_data(b_imgs_list, fn=get_imgs_fn, path=config.TRAIN.hr_img_path)
    #     b_imgs_384 = tl.prepro.threading_data(b_imgs, fn=crop_sub_imgs_fn, is_random=True)
    #     b_imgs_96 = tl.prepro.threading_data(b_imgs_384, fn=downsample_fn)

    ## If your machine have enough memory, please pre-load the whole train set.
    for idx in range(0, len(train_hr_imgs), batch_size):
        step_time = time.time()
        b_imgs_384 = tl.prepro.threading_data(train_hr_imgs[idx:idx + batch_size], fn=crop_sub_imgs_fn, is_random=True)
        b_imgs_96 = tl.prepro.threading_data(b_imgs_384, fn=downsample_fn)
        ## update G
        errM, _ = sess.run([mse_loss, g_optim_init], {t_image: b_imgs_96, t_target_image: b_imgs_384})
        print("Epoch [%2d/%2d] %4d time: %4.4fs, mse: %.8f " % (epoch, n_epoch_init, n_iter, time.time() - step_time, errM))
        total_mse_loss += errM
        n_iter += 1
    log = "[*] Epoch: [%2d/%2d] time: %4.4fs, mse: %.8f" % (epoch, n_epoch_init, time.time() - epoch_time, total_mse_loss / n_iter)
    print(log)

    ## quick evaluation on train set
    if (epoch != 0) and (epoch % 10 == 0):
        out = sess.run(net_g_test.outputs, {t_image: sample_imgs_96})  #; print('gen sub-image:', out.shape, out.min(), out.max())
        print("[*] save images")
        tl.vis.save_images(out, [ni, ni], save_dir_ginit + '/train_%d.png' % epoch)

    ## save model
    if (epoch != 0) and (epoch % 10 == 0):
        tl.files.save_npz(net_g.all_params, name=checkpoint_dir + '/g_{}_init.npz'.format(tl.global_flag['mode']), sess=sess)
